In [1]:
import requests
import json
import pandas as pd
from datetime import datetime as dt
from pmaw import PushshiftAPI
import plotly.express as px

import ipywidgets as widgets
from IPython.display import display

plot_output = widgets.Output()

def display_kw_freq(kw, subreddit, obs_hrs = 24):

    plot_output.clear_output()

    curr_obs_end = dt.timestamp(dt.now())#.replace(minute=0, second=0, microsecond=0))
    curr_obs_start = curr_obs_end - (obs_hrs * 3600)
    prev_obs_end = curr_obs_end - 86400
    prev_obs_start = curr_obs_start - 86400

    api = PushshiftAPI()
    gen = api.search_comments(q=kw, subreddit=subreddit, 
                            after=int(prev_obs_start), 
                            before=int(curr_obs_end))
    df = pd.DataFrame([result for result in gen])

    df.loc[df['created_utc'].between(curr_obs_start, curr_obs_end), 'set'] = "Today"
    df.loc[df['created_utc'].between(prev_obs_start, prev_obs_end), 'set'] = "Yesterday"
    df = df[df['set'].notnull()]
    df['created_utc'] = (df['set'] == "Yesterday") * 86400 + df['created_utc']

    df['created_dt'] = pd.to_datetime(df['created_utc'], unit='s', utc=True)
    df['created_dt'] = df['created_dt'].dt.tz_convert('Asia/Hong_Kong')

    hist = px.histogram(df, x='created_dt', color='set', barmode="overlay", #nbins=obs_hrs,
                        title=f"Occurrence of {kw} in r/{subreddit} in the last {obs_hrs} hours")
    with plot_output:
        hist.show()

keyword_text = widgets.Text(
    value='GME',
    placeholder='Type your keywords here',
    description='Keyword: '
)

subreddits = ['wallstreetbets', 'text']
subreddits_dropdown = widgets.Dropdown(
    options=subreddits,
    description='Subreddit: ')

display(keyword_text, subreddits_dropdown)

def keyword_text_eventhandler(change):
    display_kw_freq(change.new, subreddits_dropdown.value)
def subreddits_dropdown_eventhandler(change):
    display_kw_freq(keyword_text.value, change.new)
    print(change.new)

keyword_text.observe(keyword_text_eventhandler, names='value')
subreddits_dropdown.observe(subreddits_dropdown_eventhandler, names='value')

/Users/lewis/Workspace/solo/solo_bros_notebooks/.venv/lib/python3.8/site-packages/pandas/compat/__init__.py:97: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


Text(value='', description='Keyword: ', placeholder='Type your keywords here')

Dropdown(description='Subreddit: ', options=('wallstreetbets', 'text'), value='wallstreetbets')

Total:: Success Rate: 100.00% - Requests: 1 - Batches: 1
22489 results available in Pushshift
Setting limit to 22489


In [2]:
display(plot_output)

Output()